# Notebook for rink maps

## Housekeeping

### Import dependencies

In [ ]:
import pandas as pd
import numpy as np

import chickenstats

from chickenstats.chicken_nhl import Season, Scraper
from chickenstats.chicken_nhl.info import NHL_COLORS
from chickenstats.chicken_nhl.helpers import norm_coords
import chickenstats.utilities

from hockey_rink import NHLRink
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from dotenv import load_dotenv
from pathlib import Path
import os

import requests

from rich.progress import track

from sqlmodel import Table, SQLModel, Field


import datetime as dt

### Pandas options

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Environment variables

In [ ]:
env_path = Path("../.env")
load_dotenv(env_path)

### Chickenstats matplotlib style

In [ ]:
plt.style.use("chickenstats")

## Scrape data

### Schedule and game IDs

In [ ]:
season = Season(2024)

In [ ]:
schedule = season.schedule()

In [ ]:
game_ids = schedule.loc[schedule.game_state == "OFF"].game_id.tolist()  # [:10]
live_game_ids = schedule.loc[schedule.game_state == "LIVE"].game_id.tolist()

### Play-by-play

In [ ]:
scraper = Scraper(game_ids)

In [ ]:
pbp = scraper.play_by_play

In [ ]:
scraper.prep_stats(level="season")
stats = scraper.stats.reset_index(drop=True)

In [ ]:
scraper.prep_lines(level="season")
lines = scraper.lines.reset_index(drop=True)

In [ ]:
scraper.prep_team_stats(level="season")
team_stats = scraper.team_stats.reset_index(drop=True)

## Rink maps

### NHLRink

In [ ]:
rink = NHLRink(rotation=90)

### Top-6 forward line combos at 5v5

In [ ]:
team = "NSH"
strength_state = "5v5"
toi_min = 12
max_lines = 3

conds = np.logical_and.reduce(
    [lines.team == team, lines.strength_state == strength_state, lines.toi >= toi_min]
)

plot_lines = (
    lines.loc[conds]
    .sort_values(by="toi", ascending=False)
    .head(max_lines)
    .reset_index(drop=True)
)


# Setting overall figures
fig, axes = plt.subplots(nrows=2, ncols=max_lines, dpi=650, figsize=(16, 8))

fig.tight_layout(pad=1.5)

axes = axes.reshape(-1)

for row, line in plot_lines.iterrows():
    ax = axes[row]

    if row > 5:
        ax_zone = "dzone"

    else:
        ax_zone = "ozone"

    rink.draw(ax=ax, display_range=ax_zone)

    shot_events = ["MISS", "SHOT", "GOAL"]

    plot_conds = np.logical_and.reduce(
        [
            pbp.forwards == line.forwards,
            pbp.forwards_api_id == line.forwards_api_id,
            pbp.strength_state == strength_state,
            pbp.event.isin(shot_events),
        ]
    )

    plot_data = pbp.loc[plot_conds].reset_index(drop=True)

    plot_data = norm_coords(data=plot_data, norm_team=line.team)

    size_multiplier = 500

    plot_data["pred_goal_size"] = plot_data.pred_goal * size_multiplier

    for shot_event in shot_events:
        conds = np.logical_and(
            plot_data.forwards_api_id == line.forwards_api_id,
            plot_data.event == shot_event,
        )

        plot_data2 = plot_data.loc[conds]

        if plot_data2.empty:
            continue

        colors = NHL_COLORS[plot_data2.iloc[0].event_team]

        facecolor = colors[shot_event]

        if shot_event == "MISS" or shot_event == "SHOT":
            edgecolor = "#FFFFFF"

        elif shot_event == "GOAL":
            if facecolor == "#FFFFFF":
                edgecolor = colors["SHOT"]

            else:
                edgecolor = "#FFFFFF"

        rink.plot_fn(
            sns.scatterplot,
            data=plot_data2,
            x="norm_coords_x",
            y="norm_coords_y",
            color=facecolor,
            edgecolor=edgecolor,
            lw=0.75,
            s=plot_data2.pred_goal_size,
            # sizes=(0, size_multiplier),
            size_norm=(0, size_multiplier),
            zorder=100,
            alpha=0.75,
            ax=ax,
        )

    ax.set_title(
        f"{line.forwards}", x=0.5, y=1.01, ha="center", fontweight="bold", fontsize=10
    )


for row, line in plot_lines.iterrows():
    row = row + max_lines

    ax = axes[row]

    if row > max_lines - 1:
        ax_zone = "dzone"

    else:
        ax_zone = "ozone"

    rink.draw(ax=ax, display_range=ax_zone)

    shot_events = ["MISS", "SHOT", "GOAL"]

    plot_conds = np.logical_and.reduce(
        [
            pbp.opp_forwards == line.forwards,
            pbp.opp_forwards_api_id == line.forwards_api_id,
            pbp.strength_state == strength_state,
            pbp.event.isin(shot_events),
        ]
    )

    plot_data = pbp.loc[plot_conds].reset_index(drop=True)

    plot_data = norm_coords(data=plot_data, norm_team=line.team)

    size_multiplier = 500

    plot_data["pred_goal_size"] = plot_data.pred_goal * size_multiplier

    for shot_event in shot_events:
        conds = np.logical_and(
            plot_data.opp_forwards_api_id == line.forwards_api_id,
            plot_data.event == shot_event,
        )

        plot_data2 = plot_data.loc[conds]

        if plot_data2.empty:
            continue

        colors = NHL_COLORS[plot_data2.iloc[0].opp_team]

        facecolor = colors[shot_event]

        if shot_event == "MISS" or shot_event == "SHOT":
            edgecolor = "#FFFFFF"

        elif shot_event == "GOAL":
            if facecolor == "#FFFFFF":
                edgecolor = colors["SHOT"]

            else:
                edgecolor = "#FFFFFF"

        rink.plot_fn(
            sns.scatterplot,
            data=plot_data2,
            x="norm_coords_x",
            y="norm_coords_y",
            color=facecolor,
            edgecolor=edgecolor,
            lw=0.75,
            s=plot_data2.pred_goal_size,
            # sizes=(0, size_multiplier),
            size_norm=(0, size_multiplier),
            zorder=100,
            alpha=0.75,
            ax=ax,
        )


fig_title = "Nashville forward lines aren't converting 5v5 scoring chances"

fig.suptitle(fig_title, x=0.5, y=1.1, fontweight="bold", fontsize=14)

subtitle = "NSH top-3 forward line combinations by 5v5 TOI | 5v5 unblocked shot attempts for & against, sized for xG"
fig.text(s=subtitle, x=0.5, y=1.05, fontsize=12, ha="center")

attribution = "Data & xG model @chickenandstats | Viz @chickenandstats"
fig.text(s=attribution, x=0.95, y=-0.02, fontsize=12, ha="right", style="italic")

savepath = Path(f"./charts/5v5_lines.png")
# fig.savefig(savepath, transparent=False, bbox_inches="tight")